# S04 T01: Transformació Registre Log amb Regular expressions
##### L'anàlisi de registres és una funció important per al control i l'alerta, el compliment de les polítiques de seguretat, l'auditoria i el compliment normatiu, la resposta a incidents de seguretat i fins i tot les investigacions forenses. En analitzar les dades de registre, les empreses poden identificar més fàcilment les possibles amenaces i altres problemes, trobar la causa arrel i iniciar una resposta ràpida per mitigar els riscos.

##### L'analista ha d'assegurar-se que els registres consisteixen en una gamma completa de missatges i s'interpreten segons el context. Els elements de registre han d'estandaritzar-se, utilitzant els mateixos termes o terminologia, per evitar confusions i proporcionar cohesió.

##### Com Científic de Dades se t'ha proporcionat accés als registres-Logs on queda registrada l'activitat de totes les visites a realitzades a la pàgina web de l'agència de viatges "akumenius.com".

### Ex1
##### Estandaritza, identifica i enumera cada un dels atributs / variables de l'estructura de l'arxiu "Web_access_log-akumenius.com" que trobaràs al repositori de GitHub "Data-sources".

In [105]:
%matplotlib inline
import numpy as np
import pandas as pd
import re


#### VARIABLES:

#### 1- IP = www.akumenius.com + espai + IP + espai - -
###### Després de la URL de akumenius podem localitzar la IP des de la que es fa la petició





#### 2- - - [TIMESTAMP]
##### Entre claudàtors el timestamp quan es fa la petició

#### 3- POST/GET = ] + _ +" + GET/POST + espai + /
##### Si és un POST/GET

#### 4- Path = primera / + espai + HTPP/1.1"

#### 5- resposta server = HTPP/1.1" + _ + Response + _"

### Ex2
##### Neteja, preprocesa, estructura i transforma (dataframe) les dades del registre d'Accés a la web.

In [106]:
pd.set_option('display.max_colwidth', None)
df = pd.read_csv("Web_access_log-akumenius.com.txt", delimiter = "\t", header=None, index_col=False, names = ['raw'])
print(df.loc[25:31,:])
# df.style.set_properties(**{'text-align': 'left'})

                                                                                                                                                                                                                                                                                                                  raw
25                                       www.akumenius.com 66.249.76.216 - - [23/Feb/2014:03:10:38 +0100] "GET /hoteles-baratos/ofertas-hotel-Kensington-Townhouse-London-en-Kensington-54356b-destinos.html HTTP/1.1" 404 3100 "-" "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)" VLOG=-
26                                                www.akumenius.com 66.249.76.216 - - [23/Feb/2014:03:10:39 +0100] "GET /destinos-baratos/destinosEstrelles/hoteles-en-Invermere_CANAD%C3%81-con-4-estrellas HTTP/1.1" 200 8811 "-" "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)" VLOG=-
27  www.akumenius.com 66.249.76.216 - - [23/Feb/2014:03:10:40 +0100] "

In [114]:
df['web'] = df['raw'].str.split(' ').str[0]
df[['web']].iloc[46:48]

,web
46,www.akumenius.com
47,www.akumenius.com


In [108]:
df['ip'] = df['raw'].str.extract('(\d\d.\d\d\d.\d\d.\d\d\d)', expand=True)
df[['web', 'ip']].iloc[46:48]

,web,ip
46,www.akumenius.com,66.249.76.216
47,www.akumenius.com,66.249.76.216


In [109]:
# regex_timestamp="\d{1,2}/[a-zA-Z]{3}/\d{2,4}:\d{2}:\d{2}:\d{2}\s(\+|\-)\d{4}"
df["timestamp"]=df['raw'].str.extract(r'(\[.*)\]', expand=False).str.strip()
df["timestamp"] = df["timestamp"].str.replace("[", "", regex=True)
df[['web','ip','timestamp']].iloc[46:48]

,web,ip,timestamp
46,www.akumenius.com,66.249.76.216,23/Feb/2014:03:11:09 +0100
47,www.akumenius.com,66.249.76.216,23/Feb/2014:03:11:10 +0100


In [110]:
# split timestamp
from datetime import datetime
df["date"] = df["timestamp"].apply(lambda x:x.split(':',1)[0])
df["hour"] = df["timestamp"].apply(lambda x:x.split(':',1)[1]).apply(lambda x:x.split(' ',1)[0])
df["time_zone"] = df["timestamp"].apply(lambda x:x.split(' ',1)[1])
df[['timestamp','date','hour','time_zone']].iloc[46:48]

,timestamp,date,hour,time_zone
46,23/Feb/2014:03:11:09 +0100,23/Feb/2014,03:11:09,+0100
47,23/Feb/2014:03:11:10 +0100,23/Feb/2014,03:11:10,+0100


In [111]:
pattern= '(?<=\")(.*)(?=\s\/)'
df['request'] = df['raw'].apply(lambda x: re.findall(pattern,x))
df[['web','ip','timestamp', 'request']].iloc[[46,11090]]

,web,ip,timestamp,request
46,www.akumenius.com,66.249.76.216,23/Feb/2014:03:11:09 +0100,[GET]
11090,www.akumenius.com,NaN,23/Feb/2014:11:00:05 +0100,[POST]


In [112]:
pattern= '(?<=\s/)(.*)(?=\sHTTP)'
df['url'] = df['raw'].apply(lambda x: re.findall(pattern,x))
df[['web','ip','timestamp', 'request','url']].iloc[46:48]

,web,ip,timestamp,request,url
46,www.akumenius.com,66.249.76.216,23/Feb/2014:03:11:09 +0100,[GET],[destinos-baratos/destinosEstrelles/hoteles-en-Wainwright_CANAD%C3%81-con-1-estrella]
47,www.akumenius.com,66.249.76.216,23/Feb/2014:03:11:10 +0100,[GET],[hoteles-baratos/ofertas-hotel-Kenia-Nevada-en-Pradollano-2717b-destinos.html]


In [113]:
pattern= '(?<=HTTP\/1.1\"\s)(.*)(?=\s\"-")'
df['response'] = df['raw'].apply(lambda x: re.findall(pattern,x))
df[['web','ip','timestamp', 'request','url','response']].iloc[46:48]

,web,ip,timestamp,request,url,response
46,www.akumenius.com,66.249.76.216,23/Feb/2014:03:11:09 +0100,[GET],[destinos-baratos/destinosEstrelles/hoteles-en-Wainwright_CANAD%C3%81-con-1-estrella],[200 8807]
47,www.akumenius.com,66.249.76.216,23/Feb/2014:03:11:10 +0100,[GET],[hoteles-baratos/ofertas-hotel-Kenia-Nevada-en-Pradollano-2717b-destinos.html],[404 3100]


### Ex3  
##### Mostra'm la teva creativitat, Sorprèn-me fes un pas més enllà amb l'anàlisi anterior.